In [1]:
import os
import open3d as o3d
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

# switch to cpu in case of error
# from open3d.cuda.pybind.utility import Vector3dVector
from open3d.cuda.pybind.utility import Vector3dVector

from tqdm import tqdm
from scipy.spatial.transform import Rotation as R

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [8]:
# 00

velodyne_path = "/media/hty/T7/Kitti/lidar/sequences/00/pcd/"
velodyne_seg_path = "/media/hty/T7/Kitti/lidar/sequences/00/pcd_seg/"
camera_path = "/media/hty/T7/Kitti/dataset/sequences/00/image_seg/seg/"
pcd_paths = os.listdir(velodyne_path)
pcd_paths.sort()
png_paths = os.listdir(camera_path)
png_paths.sort()

with open('samples/poses/00.txt') as f:
    pose_txt = f.readlines()

poses = []
pose_index =  []
for j in range(0,1000):
    pose = np.array([float(i) for i in pose_txt[j].strip('\n').split(' ')]).reshape(3,4)
    pose_index.append(j)
    poses.append(pose)

with open("samples/calib/00/calib.txt") as f:
    iPx = f.readlines()

In [9]:
# 09
velodyne_seg_path = "/media/hty/T7/Kitti/lidar/sequences/09/pcd_seg/"
camera_path = "/media/hty/T7/Kitti/dataset/sequences/09/image_seg/seg/"
pcd_paths = os.listdir(velodyne_path)
pcd_paths.sort()
png_paths = os.listdir(camera_path)
png_paths.sort()

with open('samples/poses/09.txt') as f:
    pose_txt = f.readlines()

poses = []
pose_index =  []
for j in range(0,790):
    pose = np.array([float(i) for i in pose_txt[j].strip('\n').split(' ')]).reshape(3,4)
    pose_index.append(j)
    poses.append(pose)

with open("samples/calib/09/calib.txt") as f:
    iPx = f.readlines()


In [10]:
# homography matrix
iP3 = np.array([float(i) for i in iPx[3].strip('\n').split(' ')[1:]]).reshape(3,4)
iP2 = np.array([float(i) for i in iPx[2].strip('\n').split(' ')[1:]]).reshape(3,4)

# transformation matrix
Tr = np.array([float(i) for i in iPx[4].strip('\n').split(' ')[1:]]).reshape(4,3)

R0_rect = np.array([[1, 0.009791,  -0.0029253,0],
                    [-0.00980, 0.999938, -0.0052387,0],
                    [0.002873828, 0.005267, 0.999982,0],
                    [0,0,0,1]])

Tr_velo_to_cam = np.array([[0.0077554, -0.9999694, -0.001014303, -0.007275538],
                           [0.002294056, 0.001032122, -0.9999968, -0.06324057],
                           [0.9999673, 0.007753097,  0.00230199, -0.2670414],
                           [0,0,0,1]])

# Visualize colored point clouds

In [14]:
# Visualization

base = o3d.io.read_point_cloud(velodyne_seg_path + pcd_paths[0])
base.transform(np.vstack((poses[0],np.array([0,0,0,1]))))
filter = np.mean(np.asarray(base.colors) != (0,0,0), axis = 1,dtype=bool)
base.points = Vector3dVector(np.asarray(base.points)[filter])
base.colors = Vector3dVector(np.asarray(base.colors)[filter])

vis = o3d.visualization.Visualizer()
vis.create_window()
vis.add_geometry(base)

for i in tqdm(range(len(poses[0:1000]))):
    idx = pose_index[i]
    pcd = o3d.io.read_point_cloud(velodyne_seg_path+ pcd_paths[idx])
    filter = np.mean(np.asarray(pcd.colors) != (0,0,0), axis = 1,dtype=bool)
    pcd.points = Vector3dVector(np.asarray(pcd.points)[filter])
    pcd.colors = Vector3dVector(np.asarray(pcd.colors)[filter])
    r = R.from_matrix(poses[i][0:3,0:3])
    ax, ay, az = r.as_euler('xyz', degrees=False)
    r = o3d.geometry.get_rotation_matrix_from_xyz(np.array([0,0,-ay]))
    T = np.hstack((r, np.array([poses[i][2][3],-poses[i][0][3],-poses[i][1][3]]).reshape((3,1))))
    pcd = pcd.transform(np.vstack((T,np.array([0,0,0,1]))))
    base += pcd.voxel_down_sample(voxel_size=1)
    # base.voxel_down_sample(voxel_size=1)
    vis.update_geometry(base)
    vis.poll_events()
    vis.update_renderer()
vis.destroy_window()
# o3d.visualization.draw(base)

100%|██████████| 790/790 [00:26<00:00, 29.97it/s]


[Open3D INFO] Window window_0 created.
[Open3D INFO] EGL headless mode enabled.
[Open3D INFO] ICE servers: {"stun:stun.l.google.com:19302", "turn:user:password@34.69.27.100:3478", "turn:user:password@34.69.27.100:3478?transport=tcp"}
[Open3D INFO] Set WEBRTC_STUN_SERVER environment variable add a customized WebRTC STUN server.
[Open3D INFO] WebRTC Jupyter handshake mode enabled.


KeyboardInterrupt: 

# colorize point cloud with images

for i in tqdm(range(4500)):
    pcd = o3d.io.read_point_cloud(kitti_dir + velodyne_path + pcd_paths[i])
    pcd.paint_uniform_color([0, 0, 0])
    pts = np.asarray(pcd.points)
    pts = np.hstack([pts,np.ones((len(pts),1))])
    clrs = np.asarray(pcd.colors)

    image = cv.imread(kitti_dir + camera_path + png_paths[i])
    image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    h, w, c = image.shape

    trans = iP2 @  R0_rect @ Tr_velo_to_cam
    pixel = pts@trans.T
    for idx, pix in enumerate(pixel) :
        px = int(pix[1]/pix[2])
        py = int(pix[0]/pix[2])
        if px > 0 and py > 0 and px < h and py < w and pix[2] > 0:
            clrs[idx] = image[px][py] / 255.0
    pcd.colors = Vector3dVector(clrs)
    o3d.io.write_point_cloud(kitti_dir + velodyne_path + pcd_paths[i], pcd)
    # o3d.visualization.draw(pcd)

In [10]:
# colorization
output_path = "/Volumes/T7/Kitti/lidar/sequences/09/pcd_seg/"

for i in tqdm(range(1590)):
    pcd = o3d.io.read_point_cloud(velodyne_path + pcd_paths[i])
    pcd.paint_uniform_color([0, 0, 0])
    pts = np.asarray(pcd.points)
    pts = np.hstack([pts,np.ones((len(pts),1))])
    clrs = np.asarray(pcd.colors)
    image = cv.imread(camera_path + str(i) +"_seg.jpg")
    image = cv.resize(image,(1241, 376))
    image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    h, w, c = image.shape

    trans = iP2 @  R0_rect @ Tr_velo_to_cam
    pixel = pts@trans.T
    for idx, pix in enumerate(pixel) :
        px = int(pix[1]/pix[2])
        py = int(pix[0]/pix[2])
        if px > 0 and py > 0 and px < h and py < w and pix[2] > 0:
            clrs[idx] = image[px][py] / 255.0
    pcd.colors = Vector3dVector(clrs)
    o3d.io.write_point_cloud(output_path + pcd_paths[i], pcd)
    # o3d.visualization.draw(pcd)


100%|██████████| 1590/1590 [04:30<00:00,  5.88it/s]
